In [1]:
# Importando as bibliotecas
import requests
import zipfile
import io
import os
import pandas as pd
from typing import Optional, List

/Users/yari/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
class UrbanMobilityData:
    def __init__(self, url: str, extract_dir: str):
        """
        Inicializa a classe com a URL do arquivo zip e o diretório para extração dos arquivos.

        :param url: URL do arquivo zipado do Kaggle.
        :param extract_dir: Diretório onde os arquivos serão extraídos.
        """
        self.url = url
        self.extract_dir = extract_dir
        self.file_paths = {
            "Household": os.path.join(extract_dir, "Household.csv"),
            "Person": os.path.join(extract_dir, "Person.csv"),
            "Stage": os.path.join(extract_dir, "Stage.csv"),
            "Trip": os.path.join(extract_dir, "Trip.csv")
        }

    def download_zip(self) -> Optional[bytes]:
        """
        Faz o download do arquivo zipado da URL fornecida.

        :return: Conteúdo do arquivo zipado em bytes, ou None em caso de erro.
        """
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                return response.content
            else:
                print(f"Falha ao fazer o download dos arquivos. Status code: {response.status_code}")
                return None
        except Exception as e:
            print(f"Erro ao baixar o arquivo: {e}")
            return None

    def extract_zip(self, zip_content: bytes) -> None:
        """
        Extrai o conteúdo do arquivo zipado para o diretório especificado.

        :param zip_content: Conteúdo do arquivo zipado em bytes.
        """
        try:
            with zipfile.ZipFile(io.BytesIO(zip_content), 'r') as zip_ref:
                zip_ref.extractall(self.extract_dir)
                print("Arquivos extraídos com sucesso")
                for file in zip_ref.namelist():
                    print(f"Extraído: {file}")
        except Exception as e:
            print(f"Erro ao extrair o arquivo zipado: {e}")

    def load_csv_with_error_handling(self, file_path: str, delimiter: str = ';') -> Optional[pd.DataFrame]:
        """
        Carrega um arquivo CSV em um DataFrame do pandas com tratamento de erros.

        :param file_path: Caminho para o arquivo CSV.
        :param delimiter: Delimitador do CSV.
        :return: DataFrame carregado, ou None em caso de erro.
        """
        try:
            df = pd.read_csv(file_path, sep=delimiter, on_bad_lines='skip')
            return df
        except Exception as e:
            print(f"Erro ao carregar {file_path}: {e}")
            return None

    def load_dataframes(self) -> dict[str, Optional[pd.DataFrame]]:
        """
        Carrega todos os arquivos CSV em DataFrames e retorna um dicionário com os DataFrames.

        :return: Dicionário onde as chaves são os nomes dos arquivos e os valores são os DataFrames ou None.
        """
        dataframes = {}
        for key, path in self.file_paths.items():
            dataframes[key] = self.load_csv_with_error_handling(path)
        return dataframes


In [3]:
if __name__ == "__main__":
    url = "https://www.kaggle.com/api/v1/datasets/download/danielefm/urban-mobility-survey-federal-district-brazil?datasetVersionNumber=1"
    extract_dir = "urban_mobility_data"

    # Instanciando a classe
    data_handler = UrbanMobilityData(url, extract_dir)

    # Fazendo o download do arquivo zip
    zip_content = data_handler.download_zip()
    if zip_content:
        # Extraindo o conteúdo do arquivo zip
        data_handler.extract_zip(zip_content)
        
        # Carregando os DataFrames
        dataframes = data_handler.load_dataframes()

        # Exibindo os DataFrames carregados
        print(dataframes.items())
        for name, df in dataframes.items():          
            if df is not None:
                print(f"\n{name} DataFrame:")
                print(df.columns)

Arquivos extraídos com sucesso
Extraído: Household.csv
Extraído: Person.csv
Extraído: Stage.csv
Extraído: Trip.csv
dict_items([('Household',        household_id  people_in_household  bathrooms  bedrooms  vehicles  \
0                 3                    2          2         2         1   
1                 4                    4          1         3         3   
2                 7                    3          2         3         1   
3                22                    3          4         4         2   
4                35                    3          2         3         1   
...             ...                  ...        ...       ...       ...   
19247        345420                    4          3         4         4   
19248        345421                    6          3         4         3   
19249        345422                    5          5         6         1   
19250        345423                    3          3         4         1   
19251        345424               